# How to Start the Website
All the files relating to website is stored in this repository the folder called `pythonwebserver`. The two main things inside this folder is the file called `server.py` and a folder called `static` which contains all the static assets of the website(html, javascripts and images/videos).

The website has two main components which are the database(stores all the sensors data) and web server(serve static assets, provides an endpoint for the sensors to send the request to and sets up a proxy server to retrieve data from the database).

### How to Install the Database
*Used InfluxDB version 1.5.3-1

Open terminal and call the commands:
```sh

wget https://dl.influxdata.com/influxdb/releases/influxdb-1.5.3_linux_amd64.tar.gz

tar xvfz influxdb-1.5.3_linux_amd64.tar.gz
```
All the installation files will be located in a folder called `influxdb-1.5.3-1`. The installation size can be found by the command `du -sh influxdb-1.5.3-1`, which in this current build is 70 MB.

For more information regarding other TSDB options and installation instructions go to the Jupyter Notebook file named `Python Database Scripts and Set Up Instructions`.


### How to Start the Database

The database named `InfluxDB` was used to run the website and is installed on the Raspberry Pi on the account `tanr10`.

To run the database simply run the command :

In [ ]:
!influxd

Or to run it in the background run the command:

In [ ]:
!influxd &

By default this will run the database on `localhost:8086`.

The Influx's command line interface is nice for database management.It can be accessed by the command:

In [ ]:
!influx

Inside Influx the database that was used to store the sensor data is called `sensors`, this has the measurements(similar to SQL table) called `DO`,`EC`,`temperature` and `turbidity`

For information regarding the command line interface go to the section below in this notebook with the header [InfluxDB Common Commands Used](#influxCommand)

### How to Start the Web Server

Go to the folder named `pythonwebserver`, inside there will be a file called `server.py` which contains all the code of web server. To start this run the command:

In [ ]:
!sudo FLASK_APP=server.py flask run

By default this will run on localhost:5000. However, during operation of the website for the demo this was  on `130.113.68.111:80`.This is done by running the command:

In [ ]:
!sudo FLASK_APP=server.py flask run --host=130.113.68.111 --port=80

To run in background simply run the command:

In [ ]:
!nohup sudo FLASK_APP=server.py flask run --host=130.113.68.111 --port=80 &

### How to Access the Website

If you are running the web server locally, the website can be accessed by typing in the link `http://localhost:5000/static/index.html` or 
`http://localhost:5000/static/graph.html`. 

For the demo the website was accessed from `http://www.cas.mcmaster.ca/ollie/static/index.html` or `http://www.cas.mcmaster.ca/ollie/static/graph.html`. This is when the server is set to run on `130.113.68.111:80`

### Tips on Debugging

* If nothing is showing, check the steps above again to make sure that the web server is running
* If graph is not showing, make sure that the database is running (or still has the data inside it)
* If using chrome right click anywhere on the webpage and select inspect. Then right click on the refresh button and select the option `Empty Cache and Hard Reload`
* Right click on the webpage and click inspect. Then, go to the console tab to see if there are any errors
    * By default, there are already 2 errors, one is GET 404 NOT FOUND and another is Uncaught TypeError material-dashboard(from the front end framework). Ignore these errors and see if there any other errors.
* If graph is not showing, go to the network tab after clicking inspect then Empty Cache and Hard Reload the webpage. Then see if the database query(it looks like "query?db=...&q=...")  is present and working fine (this is the data obtained from the database that is fed into the grapher).

# Web Server

One purpose for a web server in our application is to fetch the sensors' data from the database and to provide the visualization of said data which includes an interactive map and graph. Another purpose would be to receive sensor data from a Arduino and sending that data to the database. Eventhough data from sensors could be sent directly to the database through HTTP requests, this method is far superior since this allows us to interchange different database without having to change anything on the Arduino(sensor) side, all with minimal overhead.

We have chosen to use Python for the web server because all mentioned databases have Python libraries in which we have made scripts for, thus ensuring that compatibility will not be an issue. The web server framework that we used is Flask, this is because it is popular, fast and heavily documented.

### Flask Set up
[Official Documentation](http://flask.pocoo.org/)

Simply run the command `pip install flask` to install. Once installed, you must set your python web server file as an environment variable done by running the command `FLASK_APP=server.py flask run` where in this case server.py is the filename. By default, this should run the webserver on `localhost:5000`.

Note: To run on Windows run the command `set FLASK_APP=server.py` and then `flask run`.


### HTTP Request Format to Send Sensor Data to Database(sendData endpoint)

This is the HTTP request format example that the web server takes from the Arduino(GET request)

localhost:5000/sendData?numData=2&type1=...&id1=...&value1=...&type2=...&id2=..&value2=...

Note: Change localhost:5000 accordingly.You can have any number of datapoints as you would like as specified by numData.

* numData is the number of data points that will be given

* idX is the sensor ID. We need a sensor ID because the hardware setup is as follows. There are multiple nodes each connected to a sensor, these nodes will then send their data to the central Arduino Server which then sends all the data to the web server using HTTP request. Each of the nodes that send to Arduino should be numbered to know which data comes from which node, this number is the sensor ID.

* type is the type of measurement done, for example `DO`,`EC`,`temperature` and `turbidity`

* value is the value that the sensor outputs

At the time of writing this document, the person working on the Arduino and sensors was not able to send a timestamp of when the sensor data was collected, thus a timestamp query field is not present. In the future, this can be easily added by modifying the sendData endpoint in the file `server.py` to add a variable that takes the query from the URL. Currently, the data is timestamped automatically by InfluxDB when inserted to the database.

##### Reasoning behind the format: 

Sending data with a GET request in this format is inspired by [ThingSpeak](https://thingspeak.com/), which is a website that provides a service that allows sending sensor data through HTTP requests. The person working on the Arduino prefers the format that GET request offers, since it is shorter and easier to do because for our purposes we are only sending only up to two sensor data at a time. Also, ThingSpeak provides a way to send data through POST requests, however,  in this case one must send data in a JSON formatted body. If you want to change the server to be able to receive POST request change the sendData endpoint decorator to `@app.route("/sendData",methods =["POST"])`, and change the code inside it to parse a JSON body accordingly instead of getting the query parameters. 

In [ ]:
#end point for sensors sending data to the database
@app.route("/sendData")
def data():
    numData = request.args.get('numData') # number of data to be sent
    client = influxdb.InfluxDBClient(host='localhost', port='8086',database='sensors') #change according to your database

    for i in range (int(numData)):
        #parsing the query parameters
        sensorType = request.args.get('type'+str(i+1)) #ammonia, ph etc, should equate to measurement name in influxdb
        sensorID = request.args.get('id'+str(i+1)) #which sensor is sending it, should be a tag in influxdb
        value = request.args.get('value'+str(i+1)) #sensor value

        data="%s,sensorID=%s value=%f "%(sensorType,sensorID,float(value))
        client.write(data, params={'db': client._database}, protocol='line')

    return Response("", status=200, mimetype='text/plain ')

<a id='proxy'></a>
### The Proxy server (getData endpoint)
The proxy server is set up to retrieve data from the database locally. This was done because `ollie` only allowed one port, thus we could not set up an entry to the database from another port. By setting up the proxy server, both sending data and receiving data is done through one port which was port 80.

The code below is the proxy server set up. In the file `../static/graph.js`, the data is requested to the website by the lines: 

In [ ]:
%%javascript
address='http://www.cas.mcmaster.ca/ollie/getData/query?'
query = encodeURI("db="+dbname+"&q=select value from " + sensorType + "&epoch=ns") 
url= address + query

Notice that `address` is not localhost:8086(database), this is because of the limit in the number of ports mentioned earlier. This proxy server takes everything after `getData` in the URL and concatenates it with `localhost:8086` (the port which the database runs on). Next, a GET request is made with this new URL which obtains the data from the database locally, then it streams the response back to the original URL with the address `http://www.cas.mcmaster.ca/ollie/getData/query?...` which the graph takes.

In [ ]:
#FLASK proxy server to retrieve data from influx
SITE_NAME = 'http://localhost:8086' #since this is the influxdb default port
@app.route('/getData/<path:path>')
def proxy(path):
    r=get("%s/%s"%(SITE_NAME,path), stream=True , params = request.args) #makes get request to localhost

    CHUNK_SIZE=1024
    
    #streams data obtained from database to original URL
    def generate():
        for chunk in r.iter_content(CHUNK_SIZE):
            yield chunk
    return Response(generate())

# Static Assets of the Website
Static assets that were used in this website include html files,javascript files, and images/videos, all of this is located in `pythonwebserver/static`.

### Front-End Framework
The front end framework that is used for this website called [material-dashboard](https://github.com/creativetimofficial/material-dashboard). This was chosen because of the it's clean looking aesthetics that follow Google's Material Design. This is more of a personal preference more than anything.

### Map

The map data used in this website is provided by [OpenStreetMap](https://www.openstreetmap.org/). This was chosen because the maps are detailed,accurate and most importantly it is free of charge and does not impose an limit of the number of API calls per day.

The file that controls the maps is called `map.js`. Inside this file I used a Javascript library called [Leaflet](https://leafletjs.com/) to provide the layer of interactivity of the map. Leaflet was used beacuse it contains all features that we need including having a marker that can open a popup when clicked on.

### Graph

The grahing tool that was used is provided by [Highcharts](https://www.highcharts.com/). This was chosen because of its features such as being able to interactively set custom zoom-levels, exporting data to CSV and updating live data. It is important to note that eventhough Highcharts is the provider of the graph, the product that we actually use is called Highstocks. This is because Highstocks has more features than the Highcharts such as the scrollable zoom bar in the bottom of the graph.

All code relating to the graph is located in `graph.js`, here it provides all the settings for the chart. The sensor data of the file is requested from the database using their HTTP API.

<a id='dataFormatting'></a>
#### Data formatting for Graph
As mentioned [earlier](#proxy), the proxy server gets the data locally from `localhost:8086`(InfluxDB) and streams it back to the `http://www.cas.mcmaster.ca/ollie/getData/query?...` URL. The output from InfluxDB looks like: 

This is from `http://www.cas.mcmaster.ca/ollie/getData/query?db=sensors&q=db=sensors&q=select%20value%20from%20EC&epoch=ns`

`
{"results":[{"statement_id":0,"series":[{"name":"EC","columns":["time","value"],"values":[[1532550784111188883,55.53],[1532550847057527249,56.04],[1532550909230600102,57.79],[1532550972002963491,55.97],[1532551034793243241,56.84],[1532551098022970541,57.63],[1532551161264270793,57.07],[1532551224451771663,55.62],[1532551287876458825,57.77],[1532551350563084278,54.78],[1532551413162640552,56.52],[1532551475951641437,57.62],...
`

To feed the data into the graph, values must be X-Y pairs, the code below obtained from `graph.js` parses the X and Y values from the JSON formatted output of InfluxDB and outputs it as an array called `data` which the grapher takes.

In [ ]:
%%javascript
  $.getJSON(url,
   function (data) {
     lastTime = (data.results[0].series[0].values[data.results[0].series[0].values.length - 1][0]) //the latest timestamp of data pulled
     //formats the data to x,y value pairs
     data = data.results[0].series[0].values.map(function (datapoint){
       return [
         Math.ceil((datapoint[0])/1000000), //convert ns to ms precision for javascript
         datapoint[1]
       ]
     })
      

#### Updating the Graph Dynamically

The graph initially pulls all the data inside a specified database (as shown in the function [Data Formatting](#dataFormatting)). While the graph is open on the client's browser, the code below makes a query to the database every specified time (which in this case is 10 seconds) to check whether new point(s) have arrived in the database. Then, to pull solely those new points (saving bandwidth) and graphing them.

In [ ]:
%%javascript
load: function () {

                // set up the updating of the chart for a new datapoint checks database every second
                var series = this.series[0];
                updateTaskID=setInterval(function () {

                        //getting the next value that was placed in database since last pull
                        query = encodeURI("db="+dbname+"&q=select value from " + sensorType+ " where time>" + lastTime +"&epoch=ns") //for running on pi

                        url = address + query
                        $.getJSON(url,
                         function (data) {
                           data.results[0].series[0].values.forEach(function (datapoint){
                               x=Math.ceil((datapoint[0])/1000000),
                               y=datapoint[1]
                               //add points to the graph
                               series.addPoint([x, y], true, false);
                               lastTime=datapoint[0] //the latest timestamp of data pulled
                           })

                         })
                }, 10000); //checks every 10 seconds

<a id='influxCommand'></a>
# Common InfluxDB Command Used


You can use the InfluxDB command line tool(as mentioned earlier) to see and manipulate the database using the [InfluxDB's query language](https://docs.influxdata.com/influxdb/v1.6/query_language/) (a language similar to SQL). These are some common commands used that is very useful.

### Showing all Database
SHOW DATABASES

Output if using the Raspberry Pi used for testing:

`
name:
_internal
sensors
`

Note: _internal is the default database(stores internal runtime metrics). Sensors is the database containing all sensor data, the one used for the website.

### Creating a Database
CREATE DATABASE `database name`

### Deleting a Database
DROP DATABASE `database name`

### Selecting a Database
USE `database name`

### Showing All Measurements Inside a Database

Note: Must specify the database first with the `USE` command

SHOW MEASUREMENTS 

Output if using the Raspberry Pi used for testing:

`
name:
DO
EC
temperature
turbidity
`

These measurements can be thought of as an SQL-table.

### Deleting a Measurement

DROP MEASUREMENT `measurement name`

### Showing All Data Inside a Measurement 

SELECT * FROM `measurement name`

Example output if looking into the `temperature` measurement:

SELECT * FROM temperature

`
name: temperature
time                sensorID value 
1532547558681744451 02       25
1532547561599920758 02       27
1532547575348647913 02       26.5
`

This can be useful to check whether or not the data sent from the sensor successfully arrived at the database. The most recent data will be at the bottom of the table.


### Testing the Data Insertion
Execute the following code cell, on the server side, you can check the data by executing the following commands:

1. `influx` 
2. `use MacWater`
3. `select * from sensorinfo`

In [ ]:
import os
import random

ids=["12345", "12346", "12347", "12348", "12349"] #string 
measurements = ["Temperature", "Turbidity"] # string "Dissolved%20Oxygen" "Electronical%20Conductivity"
value=0 #int
#curl -X POST "http://www.cas.mcmaster.ca/ollie/sensor/data?id=12346&measurement=Dissolved%20Oxygen&value=10"
for i in ids:
    for m in mesurements:
        cmd = 'curl -X POST \"http://www.cas.mcmaster.ca/ollie/sensor/data?id={}&measurement={}&value={}\"'.format(i,m,random.randint(1,50))
        os.system(cmd)